# Description

This notebook is representing the pipeline that was used to achive generation of multiple models whose output were ensembled at later stage. Models were based on ResNet 50. First we trained FC layer for 1 eph, and then we trained ResNet50-Layer4 for 5 eph.

In [1]:
from PIL import Image
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
import torch
from torch.optim.optimizer import Optimizer, required

import time
import zipfile
import datetime
from shutil import copyfile
from torchsummary import summary
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import os

# Data Copying
Since we were running everything on Google Collab, this part would ensure that the data got copied from the mounted Google Drive to the Collab file system.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
groot = '/content/drive/My Drive/AML'
data_zip = 'custom_split_gauss.zip'

In [ ]:
#Data copy procedure

os.mkdir("data")
copyfile(groot+"/"+data_zip, "custom_split_gauss.zip")
with zipfile.ZipFile(data_zip,"r") as zip_ref:
    zip_ref.extractall("data")

# Data Loding and Transformations
This part was used for loding of the data into the pytorch data structres, as well as for defining PCA Lighting data augmentation obtained from the referenced GitHub repository.

In [7]:
# Applying Transforms to the Data
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(size=224, pad_if_needed = True, padding_mode='reflect'),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [10]:
#General PyTorch code structre achived by following tutorials of OpenCV
# Load the Data

dataset='food_challenge'
 
# Set train and valid directory paths
train_directory = 'data/custom_split/train'
valid_directory = 'data/custom_split/validation'
 
# Batch size
bs = 32
 
# Number of classes
num_classes = 80
 
# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
}
 
# Batch size
bs = 32

# Number of classes
num_classes = len(os.listdir(valid_directory))-1  
print(num_classes)

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
# print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=True)

In [11]:
def train_and_validate(model, loss_criterion, optimizer, curIteration, scheduler, epochs=25, train_fc=True, save_at_epoch=4):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Last trained model
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(train_data_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
                    
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(valid_data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average validation loss and validation accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
                
        epoch_end = time.time()
        
        # Update scheduler step for epoch
        if(scheduler != None):
            print("Scheduler step")
            scheduler.step()
        
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
        
        # Save the model
        if train_fc:
            torch.save(model, groot+"/hrvoje/pt/"+dataset+'_model_'+str(curIteration)+'.pt')
        else:
            if(epoch == save_at_epoch):
                torch.save(model, groot+"/hrvoje/pt/"+dataset+'_model_conv_0_'+str(curIteration)+ '.pt') 
            
    return model, history

# Model Training & Validation

Training for fully connected layer

In [13]:
print("Starting train validation at:", str(datetime.datetime.now()))

for i in range(10):
    resnet50 = models.resnet50(pretrained=True)
    resnet50 = resnet50.to('cuda:0')
    for param in resnet50.parameters(): 
        param.requires_grad = False

    fc_inputs = resnet50.fc.in_features
 
    resnet50.fc = nn.Sequential(
        nn.Linear(fc_inputs, 256),  
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 80), 
        nn.LogSoftmax(dim=1) # For using NLLLoss()
    )

    resnet50 = resnet50.to('cuda:0')

    loss_func = nn.NLLLoss()
    optimizer = optim.Adam(resnet50.parameters(), lr=0.0001, weight_decay=0.0001)

    scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    num_epochs = 1
    trained_model, history = train_and_validate(resnet50, loss_func, optimizer, i, scheduler, num_epochs)  

torch.save(history, groot+"/hrvoje/"+dataset+'_history.pt')

Training for convolutional layers based on the stored weighted FC layers

In [15]:
print("Starting train validation at:", str(datetime.datetime.now()))

for i in range(3,10):
    fc_trained_model = torch.load(groot+"/hrvoje/pt/food_challenge_model_" + str(i) + ".pt")
    for name, child in fc_trained_model.named_children():
        if name == 'layer4': #or name == 'layer3' #possibility to unfreeze layer 3
            for name2, params in child.named_parameters():
                params.requires_grad = True  
    
    loss_func = nn.NLLLoss()
    optimizer = optim.Adam(fc_trained_model.parameters(), lr=0.0001, weight_decay=0.0005)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.05) 

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_epochs = 5
    trained_model, history = train_and_validate(fc_trained_model, loss_func, optimizer, i, scheduler, num_epochs, train_fc=False)

torch.save(history, groot+"/hrvoje/"+dataset+'_history.pt')